In [1]:
#| default_exp classes.DomoBootstrap

In [2]:
#| exporti
from dataclasses import dataclass, field
from fastcore.basics import patch_to
import httpx
import asyncio

import domolibrary.classes.DomoPage as dmpg

import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.Logger as lc
import domolibrary.routes.bootstrap as bootstrap_routes


In [3]:
#| export
@dataclass
class DomoBootstrap_Feature:
    id: int
    name: str
    label: str
    type: str
    purchased: bool
    enabled: bool

    @classmethod
    def _from_json_bootstrap(cls, json_obj: dict):
        dd = util_dd.DictDot(json_obj)

        bsf = cls(
            id=dd.id,
            name=dd.name,
            label=dd.label,
            type=dd.type,
            purchased=dd.purchased,
            enabled=dd.enabled
        )
        return bsf

In [4]:
#| export
@dataclass
class DomoBootstrap:
    auth : dmda.DomoAuth = field(repr = False)
    page_ls : list[dmpg.DomoPage] = field(default = None)
    feature_ls : list[DomoBootstrap_Feature] = field(default = None)
    

In [5]:
# | export
@patch_to(DomoBootstrap)
async def get_all(self: DomoBootstrap,
                  auth: dmda.DomoAuth = None, debug_api: bool = False):
    
    auth = auth or self.auth

    res =  await bootstrap_routes.get_bootstrap(auth=auth, debug_api=debug_api)

    return res.response


#### sample implementation of get_all

In [15]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-community', 
    domo_password = os.environ['DOJO_PASSWORD'], 
    domo_username= os.environ['DOMO_USERNAME'])

bs = DomoBootstrap( auth = full_auth)

bsr = await bs.get_all()
bsr['data'].keys()



dict_keys(['landingPageId', 'favorites', 'pages', 'conf', 'analytics', 'brandKit', 'environmentId', 'features', 'authorities', 'customerHash', 'locale', 'isWhitelabeled', 'isCustomerWhitelabeled', 'isFullWhitelabeled', 'trialEndDate', 'trialContactPhone', 'whitelabeledProductName', 'userAnonymous', 'userIsProxied', 'domoappsDomain', 'customerTier', 'companyTimeZone', 'disableAutoRefresh', 'wasTrial', 'buzzSignup', 'authorizedAppDomains', 'isDomoAppDomainWhitelistEnabled', 'configProperties', 'domoPublicUrl', 'advancedColorControls', 'gettingStartedPageEnabled', 'landingEntities', 'isSignupSingleSolutionInstance', 'signupApp', 'cardPageMaxConcurrentRenders', 'version'])

In [7]:
# | exporti
@patch_to(DomoBootstrap)
async def get_pages(self: DomoBootstrap,
                    auth: dmda.DomoAuth = None, return_raw : bool = False, debug_api: bool = False) -> list[dmpg.DomoPage]:

    auth = auth or self.auth

    res = await bootstrap_routes.get_bootstrap_pages(auth=auth,
                                                     debug_api=debug_api)
    
    if return_raw:
        return res.response
        
    if not res.is_success:
        return None
    
    page_ls = res.response

    self.page_ls =  await asyncio.gather(*[dmpg.DomoPage._from_bootstrap(page_obj, auth = auth) for page_obj in page_ls])

    return self.page_ls


#### sample implementation of get_pages

In [8]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-community', 
    domo_password=os.environ['DOJO_PASSWORD'],
    domo_username= os.environ['DOMO_USERNAME'])

bs = DomoBootstrap(auth=full_auth)

page_ls = await bs.get_pages(return_raw = False)

# bs.page_ls
pd.DataFrame(page_ls[10:14]).drop(columns = ['auth'] )


,id,title,parent_page_id,top_page_id,owners,cards,collections,children,is_locked,layout
0,790951325,Coronavirus (COVID-19) Dashboard,None,None,"[{'id': '1345408759', 'title': None, 'departme...",[],[],"[{'id': '1289295627', 'title': 'Dataset Schema...",None,None
1,1891382078,Fun Stuff,None,None,"[{'id': '1345408759', 'title': None, 'departme...",[],[],"[{'id': '1124898205', 'title': 'Quotes', 'pare...",None,None
2,545237196,Demo Apps,None,None,"[{'id': '826020125', 'title': None, 'departmen...",[],[],[],None,None
3,400620119,Data Science Workshop,None,None,"[{'id': '1504505648', 'title': None, 'departme...",[],[],[],None,None


In [9]:
#| exporti
@patch_to(DomoBootstrap)
async def get_features(self : DomoBootstrap,
                        auth: dmda.DomoAuth = None,
                        debug_api: bool = False,
                        return_raw:bool = False,
                        session: httpx.AsyncClient = None, 
                        ):
    
    auth = auth or self.auth

    res = await bootstrap_routes.get_bootstrap_features(auth=auth, session=session, debug_api=debug_api, return_raw=return_raw)

    if return_raw:
        return res

    feature_list = [DomoBootstrap_Feature._from_json_bootstrap(
        json_obj) for json_obj in res.response]

    return feature_list

In [10]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-community',
    domo_password=os.environ['DOJO_PASSWORD'],
    domo_username= os.environ['DOMO_USERNAME'])

bs = DomoBootstrap(auth=full_auth)

feature_ls = await bs.get_features(return_raw=False)

pd.DataFrame(feature_ls[10:14])


,id,name,label,type,purchased,enabled
0,3440,admin-pages,,PREMIUM,True,True
1,3441,admin-slideshows,,PREMIUM,True,True
2,3442,admin-access-rights,,PREMIUM,True,True
3,3443,admin-tool-downloads,,PREMIUM,True,True


In [11]:
#| exporti
@patch_to(DomoBootstrap)
async def is_group_ownership_beta(self, return_raw : bool = False):
    
    domo_feature_ls = await self.get_features(return_raw= return_raw)

    if return_raw:
        return domo_feature_ls

    match_accounts_v2 = next(
        (domo_feature for domo_feature in domo_feature_ls if domo_feature.name == 'accounts-v2'), None)

    return True if match_accounts_v2 else False

In [12]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-community',
    domo_password=os.environ['DOJO_PASSWORD'],
    domo_username= os.environ['DOMO_USERNAME'])

bs = DomoBootstrap(auth=full_auth)
await bs.is_group_ownership_beta()

False

In [13]:
#| hide
import nbdev
nbdev.nbdev_export()